### Assignment : Week 3
## Efficiently finding optimal policies in MABs

In this assignment, we will work with Multi Armed Bandit environments, and try to find the best policies using different strategies to minimize the total regret.

The aim of this exercise is to code agents capable of understanding the underlying probability distributions of the environment and finding the most optimal actions as early as possible.

You can start this assignment during/after reading Grokking Ch-4.

Let's get started!

### Imports

In [ ]:
# importing necessary stuff
import numpy as np
from pprint import pprint
from tqdm.notebook import tqdm

# if you want to use envs from Gym, import it
# import gym, gym_bandits

Let's make a simple **2-armed Bernoulli** bandit.

If you want a cleaner code, you can implement Bandits using `class` in Python.

We have included sample code for this in `bandits.py` which you can take/import.

### Generating Prob. Dist. for Bernoulli Bandit

In [ ]:
# generating the underlying probability distribution

probs = np.random.random(2)
# for action 0, reward is +1 with chance probs[0] and 0 with chance 1-probs[0], and similarly for action 1
print(probs) # however, probs is not known to the agent

### MAB Function

In [ ]:
# our MDP is a function which takes an action and returns a reward
def mab_2_env(action):
    gen = np.random.random()
    # for bernoulli bandits, the reward is 1 if the random number is less than the probability of success, else 0
    return gen < probs[action]

## Strategies

### 1. Pure Exploration

In [ ]:
# strategy returns a random action
pure_exploration = lambda Q : np.random.randint(len(Q))

### 2. Epsilon Greedy

In [ ]:
def epsilon_greedy(Q, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(len(Q))
    else:
        return np.argmax(Q)

### 3. Exponentially Decaying Epsilon Greedy

In [ ]:
def exponentially_decaying_epsilon_greedy(Q, epsilon, episode, gamma):
    epsilon = epsilon * (gamma ** episode)

    if np.random.random() < epsilon:
        return np.random.randint(len(Q))
    else:
        return np.argmax(Q)

### 4. Softmax

In [ ]:
def softmax(Q, temperature):

    # probs_list = []
    list_of_probs = []
    sum_of_exp = 0

    for i in range(len(Q)):
        exp = np.exp(Q[i]/temperature)
        list_of_probs.append(exp)
        sum_of_exp += exp

    for j in range(len(list_of_probs)):
        list_of_probs[j] /= sum_of_exp

    return np.random.choice(len(Q), 1, list_of_probs)

### 5. Upper Confidence Bound

In [ ]:
def ucb(Q, N, c):

    # sum(N) yields the number of episodes happened till now
    s = sum(N)

    U = dict()

    if s < len(Q):
        return Q[int(s)]
    else:
        for action in range(len(Q)):
            U[action] = Q[action] + c*np.sqrt(np.log(s)/N[action])
    
        return np.argmax(U)

### 6. Thompson Sampling

In [ ]:
# thompson sampling strategy

def thompson_sampling(Q, N, alpha, beta):
    samples = np.random.normal(loc=Q, scale = alpha/(np.sqrt(N) + beta))
    return np.argmax(samples)

## Calculating Regret to compare Strategies

Recall that, the regret $\mathcal{T}$ is given by,

$$\mathcal{T}=\sum  _{e=1} ^{E} \mathbb{E} \left[ v_* - q_* \left( A_e \right) \right]$$

We can only calculate it when we have the $v_*$ and $q_*$ functions known beforehand. Since we are making the MDPs from scratch, that's not an issue for us right now.

But remember, in real-life problems, these functions are not known. Hence we must be aware of multiple policy finding strategies and try the one which gives best results fastest.

In [ ]:
# function to calculate total regret

# true expected reward of the optimal action = reward[argmax(probs)] * probs[argmax(probs)]

def regret(reward, action, probs):
    # reward is the actual reward (here, +1), actions are 0 to n - 1, probs[i] = prob(getting reward on action i)
    optimal_expected_reward = np.max(probs) * reward
    expected_reward_on_action = probs[action] * reward
    return optimal_expected_reward - expected_reward_on_action

## Testing Strategies

In [ ]:
# strategy function takes in the environment function, number of actions, and a selector function
# it also takes in the number of episodes to run the strategy for (higher episodes = more accurate Q values)

def test_strategy(env, n_actions, strategy_name, n_times = 10, n_episodes = 1000, 
                  eg_epsilon = 0.01, ed_epsilon = 1, ed_gamma = 0.1, 
                  sft_temp = 1000, ucb_c = 2, thmp_alpha = 1, 
                  thmp_beta = 0, reward = 1):
    
    avg_regret = 0
    
    for _ in range(n_times):
        # return if it found the optimal action, and also the total regret
        
        # env is the mdp function, selector is the function which selects an action given the Q dict
        
        # initialize Q and N to 0s
        Q = np.zeros(n_actions)
        N = np.zeros(n_actions)
    
        total_regret = 0
    
        # loop for n_episodes
        for e in tqdm(range(n_episodes)):
            
            # selector function takes in current Q and returns an action
            # modify the selector function according to the strategy
            if strategy_name == "pure exploration":
                action = pure_exploration(Q)
            elif strategy_name == "epsilon greedy":
                action = epsilon_greedy(Q, eg_epsilon)
            elif strategy_name == "exp decaying e greedy":
                action = exponentially_decaying_epsilon_greedy(Q, ed_epsilon, sum(N), ed_gamma)
            elif strategy_name == "softmax":
                action = softmax(Q, sft_temp)
            elif strategy_name == "ucb":
                action = ucb(Q, N, ucb_c)
            elif strategy_name == "thompson":
                action = thompson_sampling(Q, sum(N), thmp_alpha, thmp_beta)
    
            cur_regret = regret(reward, action, probs)
            total_regret += cur_regret
    
            # get the reward from the environment
            reward = env(action)
    
            # update N and Q
            N[action] += 1
            Q[action] += (reward - Q[action])/N[action]

        avg_regret += total_regret/n_times
    
        # return the best action
    return np.argmax(Q), avg_regret

## Tests

In [ ]:
test_strategy(mab_2_env, 2, "pure exploration")

In [ ]:
test_strategy(mab_2_env, 2, "epsilon greedy")

In [ ]:
test_strategy(mab_2_env, 2, "exp decaying e greedy")

In [ ]:
test_strategy(mab_2_env, 2, "softmax")

In [ ]:
test_strategy(mab_2_env, 2, "ucb")

In [ ]:
test_strategy(mab_2_env, 2, "thompson")

As you can see, it returns the optimal action. Let's check if that's indeed true.

We can do that by revealing the actual `probs` distribution.

In [ ]:
print(probs)

### Todo 0 (done)

Implement the calculation of the total regret $\mathcal{T}$ for your strategy.

To do this, you will need to store the rewards obtained each episode. Modify the `strategy` function accordingly.

### Todo 1 (done)

Now, let's implement some other selection strategies and compare their regret with the simple exploration strategy.

Note that some of these strategies involve hyperparameter(s) which need to be manually adjusted. You have to play around with the values and see which one gives you best results.

This is known as "hyperparameter tuning" and is quite commonly done while working with complex models (including neural networks). Personally, you should try out some natural values (including the ones given in the book) along with some extreme values where it is easy to manually verify the correctness of your strategy.

### Todo 2 (done)

Run each strategy for 2-armed bandit environment and compare the total regrets.

You can also try plotting the regret vs episode graph and check if it matches the expected result from Grokking

In [ ]:
# import plotting libraries

import matplotlib.pyplot as plt

### Todo 3

The 2-armed bandits might be too simple for us to actually see substantial difference in the regret of these strategies. 

Let's now create a more complicated bandit environment and replicate our results on it.

We will now implement a 10-armed Gaussian bandit. 

As required, it will have possible actions and each action will generate a reward sampled from a Gaussian distribution.

Hence, each "arm" will have a randomly generated $\mu$ and $\sigma$, and the rewards will be generated with probabilities following the $\mathcal{N}(\mu, \sigma^2)$ distribution. 

In [ ]:
# 10 arm gaussian bandit

# generate the means for each arm
means = []

# generate the variance for each arm
variances = []

In [ ]:
# our MDP is a again function which takes an action and returns a reward

def mab_10_env(action):

    # for gaussian bandits, the reward is generated from a normal distribution
    raise NotImplementedError()

### Todo 4

Test the different strategies on the 10-armed gaussian bandit and verify your results.